In [ ]:
%pip install langchain pypdf nltk sentence_transformers chromadb ollama

In [10]:
%jupyter nbextension enable --py widgetsnbextension

UsageError: Line magic function `%jupyter` not found.


### Loading Libraries


In [2]:
from langchain.text_splitter import NLTKTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.vectorstores import Chroma
from langchain.llms import Ollama
import nltk
nltk.download('punkt')



[nltk_data] Downloading package punkt to C:\Users\Marcus
[nltk_data]     Ong\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
loader = PyPDFLoader("papers\YTL Corporation Berhad_Annual Report 2023.pdf")
document = loader.load()

In [4]:
document[30]

Document(page_content='SEGMENTAL REVIEW\nHOTEL OPERATIONS\nSEGMENT OVERVIEW\nThe YTL Corp Group’s hotel management and development activities \nare undertaken primarily through its listed entity, YTL REIT, and \nthrough its wholly-owned subsidiary, YTL Hotels & Properties Sdn \nBhd (“YTL Hotels”), and its subsidiaries (“YTL Hotels Group”). As at \n30 June 2023, YTL Corp held a 56.96% stake in YTL REIT.\nOPERATIONAL REVIEW\nYTL Hotels Group\nAs countries transitioned towards the endemic phase of Covid-19, \nabundant pent-up demand for travel resulted in a marked \nimprovement in occupancy and average rates across the YTL \nHotels Group portfolio. The Group’s London hotels achieved \noccupancies in the mid to high 80s, surpassing pre-pandemic \nlevels. In Kuala Lumpur, several of the Group’s city hotels produced \ntheir best occupancies since before the pandemic of up to 87%. \nThrough the year the Group continued to see better results \nfollowing the progressive re-opening of borders an

In [5]:
text_splitter = NLTKTextSplitter()
chunks = text_splitter.split_documents(document)

In [6]:
chunks[22].page_content

'Support of the company’s ongoing digital transformation agenda \nfocused on three key areas for the year under review, namely data \nand analytics, application modernisation and cybersecurity.\n\nAs part \nof its commitment to continuous advancement, YTL PowerSeraya’s \nembrace of application modernisation included the completion of \nseveral upgrading, cloud migration and re-platforming projects over \nthe year.\n\nThese initiatives have led to improvements in system performance \nand efficiency, increased scalability, an enhanced overall security \nposture and ensured compliance for the company.\n\nRetail\nYTL PowerSeraya’s retail brand, Geneco, held a market share of \n13.5% in the electricity retail market (based on retail volume over \ntotal system demand), with a sales volume of 7,408 GWh for the \nfinancial year under review.\n\nThe retail market comprises customers \nfrom the residential, commercial and industrial sectors.\n\nIn April 2023, Geneco expanded availability of its 

In [7]:
model = SentenceTransformerEmbeddings(model_name="mukaj/fin-mpnet-base")

e:\Github Repositories\RAGTests\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
db = Chroma.from_documents(chunks,model)

In [9]:
llm = Ollama(model='finance-chat')

In [42]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
template = """"<<SYS>>You are a financial analyst, Answer my questions using only the context given, Your responses have to be precise and concise as they are very important, no yapping<</SYS>>
<context>
{context}
</context>

{input} [/INST]

PARAMETER stop "[INST]"
PARAMETER stop "[/INST]"
PARAMETER stop "<<SYS>>"
PARAMETER stop "<</SYS>>"
"""
prompt = ChatPromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(llm, prompt)

In [43]:
from langchain.chains import create_retrieval_chain

retriever = db.as_retriever(search_kwargs={"k": 5})
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [50]:
response = retrieval_chain.invoke({
    "input": ""

})
response['answer']

In [32]:
response

{'input': "What is YTL's REITs",
 'context': [Document(page_content='GROUP OVERVIEW\nThe YTL Corp Group has extensive operations in Malaysia, Singapore \nand the United Kingdom (UK), as well as businesses, investments \nand projects under development in other countries including Australia, \nFrance, Indonesia, Jordan, the Netherlands, Thailand and Vietnam.\n\nYTL Corp’s entities listed on the Main Market of Bursa Malaysia \nSecurities Berhad (“Bursa Securities”) are YTL Power International \nBerhad (“YTL Power”), YTL Hospitality REIT (“YTL REIT”) and Malayan \nCement Berhad (“MCB”).\n\nThe Group also has an investment in Starhill \nGlobal Real Estate Investment Trust (“Starhill Global REIT”), which \nis listed on the Mainboard of the SGX-ST, the Singapore stock \nexchange.\n\nANNUAL REPORT 2023\n09 STRATEGIC & PERFORMANCE REVIEWMANAGEMENT DISCUSSION & ANALYSIS', metadata={'page': 10, 'source': 'papers\\YTL Corporation Berhad_Annual Report 2023.pdf'}),
  Document(page_content='44.\n\nLI

# Running out of Tokens